---
Author: Mustapha Bouhsen <br>
[LinkedIn](https://www.linkedin.com/in/mustapha-bouhsen/)<br>
[Git](https://github.com/mus514)<br>
Date: February 2, 2024<br>
---


### Load files from Azure blob storage : Set the data location and type


In [0]:
%run Repos/mustapha.bouhsen@hec.ca/ML_Pipeline_Hub/library/daily_utilities

In [0]:
# storage_account_name = "mymlprojects"
# storage_key = "?sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-03-09T09:14:29Z&st=2024-02-03T01:14:29Z&spr=https&sig=v%2Bmvq02eWWEzGfaXqGJ%2F8BJiTJrD3PPGS4eL66SIsC8%3D"

# container_name = "prod"
# mount_point = "/mnt/prod"

# dbutils.fs.mount(
#   source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/",
#   mount_point = mount_point,
#   extra_configs = {f"fs.azure.sas.{container_name}.{storage_account_name}.blob.core.windows.net":storage_key})

In [0]:
#-----------------------------------------
# Set the raw and the prod folder paths
#-----------------------------------------
raw_folder_path = "/mnt/raw/"
prod_folder_path = "/mnt/prod/"

raw_files_paths = [file.path for file in dbutils.fs.ls(raw_folder_path)]

In [0]:
#-----------------------------------------
# The schema
#-----------------------------------------
schema = StructType([
    StructField("date", DateType(), True),
    StructField("open", FloatType(), True),
    StructField("high", FloatType(), True),
    StructField("low", FloatType(), True),
    StructField("close", FloatType(), True),
    StructField("adj_close", FloatType(), True),
    StructField("volume", FloatType(), True)
])

col_float = ["open", "high", "low", "close", "volume"]

In [0]:
df = spark.read.option("header", "True").schema(schema).csv(raw_files_paths[0])

In [0]:
def process_stocks(raw_files_paths):
    try:
        for file_path in raw_files_paths:
            stock_name = file_path.split("/")[-1].split(".")[0]
            print(f'process start for {stock_name}')
            # Load the data
            df = spark.read.option("header", "True").schema(schema).csv(file_path)

            # Add year and month to make partion
            df = df.withColumn("year", F.year(F.col("date")))
            df = df.withColumn("month", F.month(F.col("date")))

            # Temp folder to save temp parquet files
            temp_folder = prod_folder_path+"temp/"
            # Partion files in folders by year and month
            df.write.partitionBy(["year", "month"]).mode("overwrite").parquet(temp_folder)
            # Delet the succes file
            dbutils.fs.rm(temp_folder+"_SUCCESS")

            # get all files path ending with .parquet
            files_paths = get_files_paths_from_folders(temp_folder, ".parquet")
            
            # Copy parquet files to final destination
            ingest_and_transform_to_parquet(files_paths, prod_folder_path, stock_name)

            # delete the temp folder
            delete_contents_recursively(temp_folder)
            print(f'process end for {stock_name}')
            print('--------------------------------')  

    except Exception as e:
        print("The error is: ",e)
    


In [0]:
process_stocks(raw_files_paths)

process start for AAPL
The error is:  name 'file_path' is not defined
